In [89]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.formula.api import ols, mnlogit
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings('ignore')

## Loading the 3 files

These files were extracted using SAS Ondemand. \
Using the .dat file and their corresponding .sas code from NHANES website, SAS Ondemand was used to convert the .dat file into a .csv file. \
We are using those converted csv files here.

In [90]:
# Loading the datasets
adult_df = pd.read_csv('data/adult.csv')
lab_df = pd.read_csv('data/lab.csv')
examination_df = pd.read_csv('data/examination.csv')

## Column Selection
Going through the paper, we shortlist the columns required from each file for our analysis

In [91]:
columns_of_interest_adult = [
    'SEQN', #Unique Identifier
    'HSAGEIR', #AGE
    'HSSEX', #SEX - 1 for Male, 2 for Female
    'DMARETHN', #Race/Ethnicity - 1: NH-White; 2: NH-Black; 3: MA; 4: Others
    'MXPSESSR', # Session
    'WTPFEX6', # Weights
    'WTPFQX6', # Weights 2
    'HAR1', #Smoking History
    'HFF18', #Annual Household income
    'HAC5A3', #Mother had Diabetes
    'HAC5A4', #Father had Diabetes
    'HFA8R', #Education
    'HAD1', #Have Diabetes or not
    'HAD3',
    'HAD4',
    'HAD5R', #Age at which diabetes diagnosis
    'HAD6', # Taking Insulin or not
    'HAT2', # Jog/Run
    'HAT2MET', #Intensity Rating
    'HAT3S', # Times done in past month
    'HAT4', # Bicycle
    'HAT4MET', #Intensity Rating
    'HAT5S', # Times done in past month
    'HAT6', # Swimming
    'HAT6MET', #Intensity Rating
    'HAT7S', # Times done in past month
    'HAT8', # Aerobics
    'HAT8MET', #Intensity Rating
    'HAT9S', # Times done in past month
    'HAT10', # Dancing
    'HAT10MET', #Intensity Rating
    'HAT11S', # Times done in past month
    'HAT12', # Calisthenics
    'HAT12MET', #Intensity Rating
    'HAT13S', # Times done in past month
    'HAT14', # Garden work
    'HAT14MET', #Intensity Rating
    'HAT15S', # Times done in past month
    'HAT16', # Lift weights
    'HAT16MET', #Intensity Rating
    'HAT17S' # Times done in past month
]

columns_of_interest_lab = [
    'SEQN', #Unique Identifier
    'DMPSTAT', # MEC or Home-examined
    #'HXPSESSR', # Session - 1: Morning; 2: Afternoon: 3: Evening
    'PHPFAST', # Time since last ate
    'G1PSI', # Glucose plasma first measurement
    'G2PSI', # Glucose plasma second measurement
    'G1PCODE', # OGTT
    'G1PTIM1', # Time between two measurements
    'G1PTIM2',
    'I1PSI', # Serum Insulin first measurement
    'I2PSI', # Serum Insulin second measurement
    'I1P2PFLG', # Instrument used
    'G1P',
    'I1P'
]

columns_of_interest_examination = [
    'SEQN', #Unique Identifier
    'DMPSTAT', #MEC or Home-examined
    'BMPBMI', #BMI
    'BMPHT', # Standing Height
    'BMPSITHT', # Sitting Height
    'MAPF12R', # Pregnant : 1-Yes; 2-No
    'MYPC17', # Pregnant
    'MPPB4', # Age at menarche
    'MYPC2', # Age at menarche
    'MAPF2', # Age at menarche
    'PEPPREG', # Pregnant
    'PEPPACE', # Pacemaker
    'PEP3B1', # Arm amputee
    'PEP3B2', # Leg amputee
    'BMPWT', # Weight
    'PEP12A1' # Resistance
]

## Data Cleaning

Joining all 3 table on 'SEQN'.\
We remove those subjects that completed physical examination and laboratory test at home. \
We limit the age to 40-74, remove pregnant women, remove 'other' race/ethnicity and remove 55 participants with insulin treated diabetes diagnosed before 40.

In [92]:
# Keeping only columns that we need
adult_df = adult_df.filter(columns_of_interest_adult)
lab_df = lab_df.filter(columns_of_interest_lab)
examination_df = examination_df.filter(columns_of_interest_examination)

# Removing home-examined participants
lab_df = lab_df[lab_df['DMPSTAT']!=3].drop(columns='DMPSTAT')
examination_df = examination_df[examination_df['DMPSTAT']!=3].drop(columns='DMPSTAT')

# Merging the three different files into one dataframe on 'SEQN'
merged_df = pd.merge(adult_df, lab_df, on='SEQN', how='inner')
merged_df = pd.merge(merged_df, examination_df, on='SEQN', how='inner')

# Limiting age to 40-74
merged_df = merged_df[(merged_df['HSAGEIR'] >= 40) & (merged_df['HSAGEIR'] <= 74)]

# Removing pregnant women
merged_df = merged_df[(merged_df['MAPF12R'] != 1)]

# Limiting race/ethnicity
merged_df = merged_df[(merged_df['DMARETHN'] != 4)]

# Removing participants with insulin treated diabetes diagnosed before 40
merged_df = merged_df[~((merged_df['HAD5R']<40) & (merged_df['HAD6']==1))]
merged_df.shape

(7424, 66)

## Feature Engineering

Next we do feature engineering according to how it's specified in the paper

In [93]:
#Being vigorously active was defined as participating ≥3 times/wk in an activity with a 
#metabolic equivalent (MET) level of ≥6 for participants who were aged ≥60 y and 7 for partici- pants who were <60 y. 
#The term “moderately active” was defined as participating ≥ 5 times/wk in physical activities, ≤ 2 of which were defined as vigorous. 
#“Lightly active” was defined as participation that was not “vigorously active” or “moderately active.” 
#“Sedentary” was defined as engaging in no leisure-time physical activity.

def classify_physical_activity(row):
    # Convert 'Times done in past month' to 'Times done per week'
    times_per_week = {
        'HAT3S': row['HAT3S'] / 4,
        'HAT5S': row['HAT5S'] / 4,
        'HAT7S': row['HAT7S'] / 4,
        'HAT9S': row['HAT9S'] / 4,
        'HAT11S': row['HAT11S'] / 4,
        'HAT13S': row['HAT13S'] / 4,
        'HAT15S': row['HAT15S'] / 4
    }

    # Pair up activity columns with their corresponding intensity rating columns
    activity_intensity_pairs = [
        ('HAT3S', 'HAT2MET'),
        ('HAT5S', 'HAT4MET'),
        ('HAT7S', 'HAT6MET'),
        ('HAT9S', 'HAT8MET'),
        ('HAT11S', 'HAT10MET'),
        ('HAT13S', 'HAT12MET'),
        ('HAT15S', 'HAT14MET')
    ]

    # Calculate total vigorous activities per week
    vigorous_threshold = 7 if row['HSAGEIR'] < 60 else 6
    vigorous_activities = sum(times_per_week[activity] for activity, intensity in activity_intensity_pairs if row[intensity] >= vigorous_threshold)

    # Calculate total physical activities per week
    total_activities = sum(times_per_week.values())

    # Classify physical activity level
    if vigorous_activities >= 3:
        return 'vigorously_active'
    elif total_activities >= 5 and vigorous_activities <= 2:
        return 'moderately_active'
    elif total_activities > 0:
        return 'lightly_active'
    else:
        return 'sedentary'

In [94]:
merged_df['PHPFAST'] = merged_df['PHPFAST'].replace(88888, merged_df['PHPFAST'].median())
merged_df['G1PSI'] = merged_df['G1PSI'].replace(888888, merged_df['G1PSI'].median())
merged_df['G2PSI'] = merged_df['G2PSI'].replace(888888, merged_df['G2PSI'].median())
median_value = merged_df['G2PSI'].median()
merged_df['G2PSI'] = merged_df['G2PSI'].fillna(median_value)

#merged_df['BMPHT'] = merged_df['BMPHT'].replace(88888, merged_df['BMPHT'].median())
merged_df['BMPSITHT'] = merged_df['BMPSITHT'].replace(88888, merged_df['BMPSITHT'].median())

# Making leg-length column
merged_df['leg_length'] = merged_df['BMPHT'] - merged_df['BMPSITHT']

#Making leg length to height ratio
merged_df['leg_length_to_height_ratio'] = merged_df['leg_length']/merged_df['BMPHT']

# map sex
merged_df['Sex'] = merged_df['HSSEX'].map({
    1: 'Male',
    2: 'Female'
})

# Race/ethnicity was categorized into three groups: non-Hispanic white, non-Hispanic black, and Mexican American.
merged_df['race_ethnicity'] = merged_df['DMARETHN'].map({
    1: 'non_Hispanic_white',
    2: 'non_Hispanic_black',
    3: 'Mexican_American'
})

# Subjects were classified as hav- ing parental history of diabetes if either their biological father or mother had dia- betes
merged_df['parental_diabetes'] = (merged_df['HAC5A4'] == 1) | (merged_df['HAC5A3'] == 1)

# Physical activity was classified into four categories (vigorously active, moderately active, 
# lightly active, and sedentary) based on frequency and intensity of reported physical activities 
columns_to_fill = ['HAT2', 'HAT2MET', 'HAT3S', 'HAT4', 'HAT4MET', 'HAT5S', 'HAT6', 'HAT6MET','HAT7S',
                   'HAT8','HAT8MET', 'HAT9S', 'HAT10', 'HAT10MET', 'HAT11S', 'HAT12', 'HAT12MET', 'HAT13S', 
                   'HAT14','HAT14MET', 'HAT15S', 'HAT16', 'HAT16MET', 'HAT17S']

merged_df[columns_to_fill] = merged_df[columns_to_fill].fillna(0)

# Apply the classification function to each row of the DataFrame
merged_df['physical_activity'] = merged_df.apply(classify_physical_activity, axis=1)

# Dichotomize education level at 12 years
merged_df['education_12_years'] = merged_df['HFA8R'] >= 12

# Annual household income was dichotomized at $20,000
merged_df['income_20000_or_more'] = merged_df['HFF18'].map({
    1: False, 
    2: True,
    8: False,
    9: False,
    0: False
})

# Subjects were classi- fied as “ever smokers” if they reported that they had smoked 
# at least 100 cigarettes during their lives; otherwise they were classified as “never smokers.”
merged_df['smoking_hist'] = merged_df['HAR1'].map({
    1: 'ever_smokers',
    2: 'never_smokers'
})

#Calculating weighted measurements
merged_df['Weighted_height'] = merged_df['BMPHT']*merged_df['WTPFEX6']
merged_df['Weighted_leg_length'] = merged_df['leg_length']*merged_df['WTPFEX6']
merged_df['Weighted_ratio'] = merged_df['leg_length_to_height_ratio']*merged_df['WTPFEX6']

# Calculate sex and race specific z-scores for height
#merged_df['height_z'] = merged_df.groupby(['Sex', 'race_ethnicity'])['BMPHT'].transform(lambda x: (x - merged_df['Weighted_height'].mean()) / merged_df['Weighted_height'].std())

# Calculate sex and race specific z-scores for leg length
#merged_df['leg_length_z'] = merged_df.groupby(['Sex', 'race_ethnicity'])['leg_length'].transform(lambda x: (x - merged_df['Weighted_leg_length'].mean()) / merged_df['Weighted_leg_length'].std())

# Calculate sex and race specific z-scores for leg length to height ratio
#merged_df['ratio_z'] = merged_df.groupby(['Sex', 'race_ethnicity'])['leg_length_to_height_ratio'].transform(lambda x: (x - merged_df['Weighted_ratio'].mean()) / merged_df['Weighted_ratio'].std())

anthropometric_cols = ['BMPHT', 'leg_length', 'leg_length_to_height_ratio']  # Replace with your column names

# Define the grouping variables for sex and race/ethnicity
grouping_vars = ['Sex', 'race_ethnicity']  # Replace with your column names

# Function to calculate weighted mean and standard deviation
def weighted_mean_std(group, weight_col, col):
    weights = group[weight_col]
    values = group[col]
    weighted_mean = np.average(values, weights=weights)
    weighted_std = np.sqrt(np.average((values - weighted_mean)**2, weights=weights))
    return weighted_mean, weighted_std

# Calculate weighted z-scores for each anthropometric measurement
for col in anthropometric_cols:
    z_score_col = f'{col}_z_score'
    merged_df[z_score_col] = 0.0  # Initialize z-score column with zeros
    
    for _, group in merged_df.groupby(grouping_vars):
        weight_col = 'WTPFEX6'  # Replace with your sample weight column name
        weighted_mean, weighted_std = weighted_mean_std(group, weight_col, col)
        group[z_score_col] = (group[col] - weighted_mean) / weighted_std
        
        merged_df.loc[group.index, z_score_col] = group[z_score_col]

## Adiposity

Here we will remove all those subjects for whom we can't calculate % body fat. \
Next we will calculate Fat Free Mass and use it for calculation of % body fat.

In [95]:
# Function to convert Valhalla resistance to RJL resistance
def convert_valh(row):
    if row['Sex'] == 'Male':
        rjl = 2.5 + 0.98*row['PEP12A1']
    elif row['Sex'] == 'Female':
        rjl = 9.6 + 0.96*row['PEP12A1']
    else:
        rjl = np.nan
    return rjl

In [96]:
# Function to calculate fat free mass
def calculate_ffm(row):
    if row['Sex'] == 'Male':
        ffm = (-10.678 + 0.262 * row['BMPWT'] + 0.652 * (row['BMPHT']**2/row['RJL']) + 0.015 * row['RJL'])
    elif row['Sex'] == 'Female':
        ffm = (-9.529 + 0.168 * row['BMPWT'] + 0.696 * (row['BMPHT']**2/row['RJL']) + 0.016 * row['RJL'])
    else:
        ffm = np.nan
    return ffm

In [97]:
adiposity_df = merged_df[~((merged_df['PEP12A1'].isna()) | 
                           (merged_df['PEP12A1']==8888) | 
                           (merged_df['BMPWT']==888888) | 
                           (merged_df['BMPHT'].isna()) | 
                           (merged_df['BMPHT']==88888)
                           )
                        ]

# Calculating rjl resistance 
adiposity_df['RJL'] = adiposity_df.apply(convert_valh, axis=1)

# Calculating Fat Free Mass
adiposity_df['FFM'] = adiposity_df.apply(calculate_ffm, axis=1)

# Calculating % body fat
adiposity_df['PBF'] =((adiposity_df['BMPWT'] - adiposity_df['FFM'])/(adiposity_df['BMPWT']))*100

In [98]:
adiposity_df.shape

(6778, 84)

## HOMA-IR

In [99]:
def keepforHOMA(row):
    if row['MXPSESSR'] == 1:
        if row['PHPFAST'] >= 12 and (3.0 <= row['G1PSI'] <= 25.0) and (20.0 <= row['I1PSI'] <= 400.0):
            return 1
    elif row['MXPSESSR'] == 2 or row['MXPSESSR'] == 3:
        if row['PHPFAST'] >= 6 and (3.0 <= row['G1PSI'] <= 25.0) and (20.0 <= row['I1PSI'] <= 400.0):
            return 1
    else:
        return 0
    return 0

In [100]:
# Removing those who have diabetes
HOMA_df = adiposity_df[~((adiposity_df['HAD1']==1))]

# Removing NULLs and 'Blank but applicable'
HOMA_df = HOMA_df[~((HOMA_df['PHPFAST']==88888) | (HOMA_df['PHPFAST'].isna()))]
HOMA_df = HOMA_df[~((HOMA_df['G1PSI']==888888) | (HOMA_df['G1PSI'].isna()))]
HOMA_df = HOMA_df[~((HOMA_df['G2PSI']==888888) | (HOMA_df['G2PSI'].isna()))]
HOMA_df = HOMA_df[~((HOMA_df['I1PSI']==8888888) | (HOMA_df['I1PSI'].isna()))]

# Flag to mark which rows to keep and then drop other rows
HOMA_df['keepforHOMA'] = HOMA_df.apply(keepforHOMA, axis=1)
HOMA_df = HOMA_df[HOMA_df['keepforHOMA']==1]

# Calculating HOMA-IR for all subjects
HOMA_df['HOMA_IR'] = (HOMA_df['G1PSI']*(HOMA_df['I1PSI']/6))/22.5

In [101]:
homa_test = HOMA_df[['HSAGEIR','Sex','race_ethnicity','parental_diabetes',
         'physical_activity','education_12_years','income_20000_or_more',
         'smoking_hist','MAPF2','PBF','BMPHT','leg_length','leg_length_to_height_ratio','G1PSI','I1PSI',
         'BMPHT_z_score','leg_length_z_score','leg_length_to_height_ratio_z_score']]

In [102]:
homa_test.to_csv('homa_new.csv')

In [103]:
HOMA_df = pd.read_csv('homa_Final.csv')

## OGTT

In [104]:
def fastorbldsmpl(row):
    if row['MXPSESSR'] == 1:
        if row['PHPFAST'] >= 12 and (105.0 <= row['G1PTIM1'] <= 135.0):
            return 1
    elif row['MXPSESSR'] == 2 or row['MXPSESSR'] == 3:
        if row['PHPFAST'] >= 6 and (105.0 <= row['G1PTIM1'] <= 135.0):
            return 1
    return 0 

In [105]:
# def category(row):
#     G1PSI_rounded = round(row['G1PSI'], 1)
#     G2PSI_rounded = round(row['G2PSI'], 1)

#     if row['MXPSESSR'] == 1:
#         if G1PSI_rounded < 7.8 and G2PSI_rounded < 7.8:
#             return 'normal'
#         elif G1PSI_rounded < 7.8 and (7.8 <= G2PSI_rounded <= 11.1):
#             return 'IGT'
#         elif G1PSI_rounded >= 7.8 or G2PSI_rounded >= 11.1:
#             return 'diabetes'
#         else:
#             return 'Not Valid'
#     else:
#         if G1PSI_rounded < 7.8 and G2PSI_rounded < 7.8:
#             return 'normal'
#         elif G1PSI_rounded < 7.8 and (7.8 <= G2PSI_rounded <= 13.9):
#             return 'IGT'
#         elif G1PSI_rounded >= 7.8 or G2PSI_rounded >= 13.9:
#             return 'diabetes'
#         else:
#             return 'Not Valid'

In [106]:
def category(row):
    G1PSI_rounded = round(row['G1PSI'], 1)
    G2PSI_rounded = round(row['G2PSI'], 1)

    if row['MXPSESSR'] == 1:
        if G1PSI_rounded < 7.8 and G2PSI_rounded < 7.8:
            return 'normal'
        elif G1PSI_rounded < 7.8 and G2PSI_rounded <= 11.1:
            return 'IGT'
        elif G1PSI_rounded >= 7.8 or G2PSI_rounded >= 11.1:
            return 'diabetes'
        else:
            return 'Not Valid'
    else:
        if G1PSI_rounded < 7.8 and G2PSI_rounded < 7.8:
            return 'normal'
        elif G1PSI_rounded < 7.8 and G2PSI_rounded <= 13.9:
            return 'IGT'
        elif G1PSI_rounded >= 7.8 or G2PSI_rounded >= 13.9:
            return 'diabetes'
        else:
            return 'Not Valid'

In [107]:
# Removing those who had diabetes
ogtt_df = merged_df[~(((merged_df['HAD1']==1) & (merged_df['HAD3']!=1)) | ((merged_df['HAD1']==1) & (merged_df['HAD4']==1)))]

# Removing those who did not complete OGTT
ogtt_df = ogtt_df[ogtt_df['G1PCODE'].isna()]

# Removing those who did not fast or blood samole not drawn in time
ogtt_df['keepigt'] = ogtt_df.apply(fastorbldsmpl,axis=1)
ogtt_df = ogtt_df[ogtt_df['keepigt']==1]

# Classifying into category
ogtt_df['GT_Category'] = ogtt_df.apply(category, axis=1)

# Merginf with those who have diabetes
diabetes_df = merged_df[(((merged_df['HAD1']==1) & (merged_df['HAD3']!=1)) | ((merged_df['HAD1']==1) & (merged_df['HAD4']==1)))]
diabetes_df['keepigt'] = 1
diabetes_df['GT_Category'] = 'diabetes'
ogtt_df = pd.concat([ogtt_df, diabetes_df], axis=0, ignore_index=True)

ogtt_df = ogtt_df[~(ogtt_df['BMPHT']==88888)]
ogtt_df = ogtt_df[~(ogtt_df['GT_Category']=='Not Valid')]

# Statistical Analysis

## Percentage Body Fat
We will use multivariate linear regression
## Female

In [177]:
subset_df = adiposity_df[adiposity_df['Sex']=='Female']

target = subset_df['PBF']

subset_df['MAPF2'] = subset_df['MAPF2'].fillna(0)

#std_devs = subset_df[['BMPHT','leg_length','leg_length_to_height_ratio']].std()

std_devs = subset_df[['BMPHT_z_score','leg_length_z_score','leg_length_to_height_ratio_z_score']].std()

## Height

### Model 1

In [113]:
anthropometric_measures = ['BMPHT_z_score']
model = ols('target ~ BMPHT_z_score+HSAGEIR',data=subset_df).fit()

# Calculate 1-SD lower comparison for each variable in the model
print(f"\n1-SD lower comparison for each variable with confidence intervals in the Basic model:")

# Iterate through each predictor
for predictor in anthropometric_measures:
# Coefficient for the predictor
    coef = model.params[predictor]
    
    # Standard deviation of the predictor
    std_dev = std_devs[predictor] if predictor in std_devs else 0
    
    # Calculate the effect of a 1-SD decrease in the predictor
    BMPHT_female_effect_model1 = coef * (-std_dev)
    
    # Calculate the confidence intervals for the 1-SD lower comparison
    ci_df = model.conf_int()  # Get the confidence interval DataFrame
    
    # Access the lower and upper bounds of the predictor's confidence interval
    if predictor in ci_df.index:
        BMPHT_female_lower_ci_model1 = ci_df.loc[predictor, 0] * (-std_dev)
        BMPHT_female_upper_ci_model1 = ci_df.loc[predictor, 1] * (-std_dev)
        
        # Print the effect and confidence intervals
        print(f"{predictor}: {BMPHT_female_effect_model1:.4f} change in percent body fat for 1-SD lower")
        print(f"    95% CI: [{BMPHT_female_lower_ci_model1:.4f}, {BMPHT_female_upper_ci_model1:.4f}]")
    else:
        print(f"Predictor '{predictor}' not found in the model confidence intervals.")

    sign = "+" if BMPHT_female_effect_model1 >= 0 else ""
    height_female_model1 = f"{sign}{abs(BMPHT_female_effect_model1):.2f} ({BMPHT_female_upper_ci_model1:.2f}-{BMPHT_female_lower_ci_model1:.2f})"
    print(height_female_model1)


1-SD lower comparison for each variable with confidence intervals in the Basic model:
BMPHT_z_score: 0.7971 change in percent body fat for 1-SD lower
    95% CI: [1.1712, 0.4229]
+0.80 (0.42-1.17)


### Model 2

In [114]:
model = ols('target ~ BMPHT_z_score+HSAGEIR+parental_diabetes+physical_activity+education_12_years+income_20000_or_more+smoking_hist+MAPF2',data=subset_df).fit()

# Calculate 1-SD lower comparison for each variable in the model
print(f"\n1-SD lower comparison for each variable with confidence intervals in the Basic model:")

# Iterate through each predictor
for predictor in anthropometric_measures:
# Coefficient for the predictor
    coef = model.params[predictor]
    
    # Standard deviation of the predictor
    std_dev = std_devs[predictor] if predictor in std_devs else 0
    
    # Calculate the effect of a 1-SD decrease in the predictor
    BMPHT_female_effect_model2 = coef * (-std_dev)
    
    # Calculate the confidence intervals for the 1-SD lower comparison
    ci_df = model.conf_int()  # Get the confidence interval DataFrame
    
    # Access the lower and upper bounds of the predictor's confidence interval
    if predictor in ci_df.index:
        BMPHT_female_lower_ci_model2 = ci_df.loc[predictor, 0] * (-std_dev)
        BMPHT_female_upper_ci_model2 = ci_df.loc[predictor, 1] * (-std_dev)
        
        # Print the effect and confidence intervals
        print(f"{predictor}: {BMPHT_female_effect_model2:.4f} change in percent body fat for 1-SD lower")
        print(f"    95% CI: [{BMPHT_female_lower_ci_model2:.4f}, {BMPHT_female_upper_ci_model2:.4f}]")
    else:
        print(f"Predictor '{predictor}' not found in the model confidence intervals.")

    sign = "+" if BMPHT_female_effect_model2 >= 0 else ""
    height_female_model2 = f"{sign}{abs(BMPHT_female_effect_model2):.2f} ({BMPHT_female_upper_ci_model2:.2f}-{BMPHT_female_lower_ci_model2:.2f})"
    print(height_female_model2)


1-SD lower comparison for each variable with confidence intervals in the Basic model:
BMPHT_z_score: 0.6019 change in percent body fat for 1-SD lower
    95% CI: [0.9695, 0.2343]
+0.60 (0.23-0.97)


In [115]:
height_female_model3 = '-'

## Leg Length

### Model 1

In [116]:
anthropometric_measures = ['leg_length_z_score']
model = ols('target ~ leg_length_z_score+HSAGEIR',data=subset_df).fit()

# Calculate 1-SD lower comparison for each variable in the model
print(f"\n1-SD lower comparison for each variable with confidence intervals in the Basic model:")

# Iterate through each predictor
for predictor in anthropometric_measures:
# Coefficient for the predictor
    coef = model.params[predictor]
    
    # Standard deviation of the predictor
    std_dev = std_devs[predictor] if predictor in std_devs else 0
    
    # Calculate the effect of a 1-SD decrease in the predictor
    leg_length_female_effect_model1 = coef * (-std_dev)
    
    # Calculate the confidence intervals for the 1-SD lower comparison
    ci_df = model.conf_int()  # Get the confidence interval DataFrame
    
    # Access the lower and upper bounds of the predictor's confidence interval
    if predictor in ci_df.index:
        leg_length_female_lower_ci_model1 = ci_df.loc[predictor, 0] * (-std_dev)
        leg_length_female_upper_ci_model1 = ci_df.loc[predictor, 1] * (-std_dev)
        
        # Print the effect and confidence intervals
        print(f"{predictor}: {leg_length_female_effect_model1:.4f} change in percent body fat for 1-SD lower")
        print(f"    95% CI: [{leg_length_female_lower_ci_model1:.4f}, {leg_length_female_upper_ci_model1:.4f}]")
    else:
        print(f"Predictor '{predictor}' not found in the model confidence intervals.")
    
    sign = "+" if leg_length_female_effect_model1 >= 0 else ""
    leg_length_female_model1 = f"{sign}{abs(leg_length_female_effect_model1):.2f} ({leg_length_female_upper_ci_model1:.2f}-{leg_length_female_lower_ci_model1:.2f})"
    print(leg_length_female_model1)


1-SD lower comparison for each variable with confidence intervals in the Basic model:
leg_length_z_score: 1.0789 change in percent body fat for 1-SD lower
    95% CI: [1.4515, 0.7062]
+1.08 (0.71-1.45)


### Model 2

In [117]:
model = ols('target ~ leg_length_z_score+HSAGEIR+parental_diabetes+physical_activity+education_12_years+income_20000_or_more+smoking_hist+MAPF2',data=subset_df).fit()

# Calculate 1-SD lower comparison for each variable in the model
print(f"\n1-SD lower comparison for each variable with confidence intervals in the Basic model:")

# Iterate through each predictor
for predictor in anthropometric_measures:
# Coefficient for the predictor
    coef = model.params[predictor]
    
    # Standard deviation of the predictor
    std_dev = std_devs[predictor] if predictor in std_devs else 0
    
    # Calculate the effect of a 1-SD decrease in the predictor
    leg_length_female_effect_model2 = coef * (-std_dev)
    
    # Calculate the confidence intervals for the 1-SD lower comparison
    ci_df = model.conf_int()  # Get the confidence interval DataFrame
    
    # Access the lower and upper bounds of the predictor's confidence interval
    if predictor in ci_df.index:
        leg_length_female_lower_ci_model2 = ci_df.loc[predictor, 0] * (-std_dev)
        leg_length_female_upper_ci_model2 = ci_df.loc[predictor, 1] * (-std_dev)
        
        # Print the effect and confidence intervals
        print(f"{predictor}: {leg_length_female_effect_model2:.4f} change in percent body fat for 1-SD lower")
        print(f"    95% CI: [{leg_length_female_lower_ci_model2:.4f}, {leg_length_female_upper_ci_model2:.4f}]")
    else:
        print(f"Predictor '{predictor}' not found in the model confidence intervals.")
    
    sign = "+" if leg_length_female_effect_model2 >= 0 else ""
    leg_length_female_model2 = f"{sign}{abs(leg_length_female_effect_model2):.2f} ({leg_length_female_upper_ci_model2:.2f}-{leg_length_female_lower_ci_model2:.2f})"
    print(leg_length_female_model2)


1-SD lower comparison for each variable with confidence intervals in the Basic model:
leg_length_z_score: 0.8592 change in percent body fat for 1-SD lower
    95% CI: [1.2263, 0.4920]
+0.86 (0.49-1.23)


In [118]:
leg_length_female_model3 = '-'

## Leg Length To Height Ratio

### Model 1

In [119]:
anthropometric_measures = ['leg_length_to_height_ratio_z_score']
model = ols('target ~ leg_length_to_height_ratio_z_score+HSAGEIR',data=subset_df).fit()

# Calculate 1-SD lower comparison for each variable in the model
print(f"\n1-SD lower comparison for each variable with confidence intervals in the Basic model:")

# Iterate through each predictor
for predictor in anthropometric_measures:
# Coefficient for the predictor
    coef = model.params[predictor]
    
    # Standard deviation of the predictor
    std_dev = std_devs[predictor] if predictor in std_devs else 0
    
    # Calculate the effect of a 1-SD decrease in the predictor
    leg_length_to_height_ratio_female_effect_model1 = coef * (-std_dev)
    
    # Calculate the confidence intervals for the 1-SD lower comparison
    ci_df = model.conf_int()  # Get the confidence interval DataFrame
    
    # Access the lower and upper bounds of the predictor's confidence interval
    if predictor in ci_df.index:
        leg_length_to_height_ratio_female_lower_ci_model1 = ci_df.loc[predictor, 0] * (-std_dev)
        lleg_length_to_height_ratio_female_upper_ci_model1 = ci_df.loc[predictor, 1] * (-std_dev)
        
        # Print the effect and confidence intervals
        print(f"{predictor}: {leg_length_to_height_ratio_female_effect_model1:.4f} change in percent body fat for 1-SD lower")
        print(f"    95% CI: [{leg_length_to_height_ratio_female_lower_ci_model1:.4f}, {lleg_length_to_height_ratio_female_upper_ci_model1:.4f}]")
    else:
        print(f"Predictor '{predictor}' not found in the model confidence intervals.")
    
    sign = "+" if leg_length_to_height_ratio_female_effect_model1 >= 0 else ""
    ratio_female_model1 = f"{sign}{abs(leg_length_to_height_ratio_female_effect_model1):.2f} ({lleg_length_to_height_ratio_female_upper_ci_model1:.2f}-{leg_length_to_height_ratio_female_lower_ci_model1:.2f})"
    print(ratio_female_model1)


1-SD lower comparison for each variable with confidence intervals in the Basic model:
leg_length_to_height_ratio_z_score: 1.1075 change in percent body fat for 1-SD lower
    95% CI: [1.4499, 0.7652]
+1.11 (0.77-1.45)


### Model 2

In [120]:
model = ols('target ~ leg_length_to_height_ratio_z_score+HSAGEIR+parental_diabetes+physical_activity+education_12_years+income_20000_or_more+smoking_hist+MAPF2',data=subset_df).fit()

# Calculate 1-SD lower comparison for each variable in the model
print(f"\n1-SD lower comparison for each variable with confidence intervals in the Basic model:")

# Iterate through each predictor
for predictor in anthropometric_measures:
# Coefficient for the predictor
    coef = model.params[predictor]
    
    # Standard deviation of the predictor
    std_dev = std_devs[predictor] if predictor in std_devs else 0
    
    # Calculate the effect of a 1-SD decrease in the predictor
    leg_length_to_height_ratio_female_effect_model2 = coef * (-std_dev)
    
    # Calculate the confidence intervals for the 1-SD lower comparison
    ci_df = model.conf_int()  # Get the confidence interval DataFrame
    
    # Access the lower and upper bounds of the predictor's confidence interval
    if predictor in ci_df.index:
        leg_length_to_height_ratio_female_lower_ci_model2 = ci_df.loc[predictor, 0] * (-std_dev)
        leg_length_to_height_ratio_female_upper_ci_model2 = ci_df.loc[predictor, 1] * (-std_dev)
        
        # Print the effect and confidence intervals
        print(f"{predictor}: {leg_length_to_height_ratio_female_effect_model2:.4f} change in percent body fat for 1-SD lower")
        print(f"    95% CI: [{leg_length_to_height_ratio_female_lower_ci_model2:.4f}, {leg_length_to_height_ratio_female_upper_ci_model2:.4f}]")
    else:
        print(f"Predictor '{predictor}' not found in the model confidence intervals.")
    
    sign = "+" if leg_length_to_height_ratio_female_effect_model2 >= 0 else ""
    ratio_female_model2 = f"{sign}{abs(leg_length_to_height_ratio_female_effect_model2):.2f} ({leg_length_to_height_ratio_female_upper_ci_model2:.2f}-{leg_length_to_height_ratio_female_lower_ci_model2:.2f})"
    print(ratio_female_model2)


1-SD lower comparison for each variable with confidence intervals in the Basic model:
leg_length_to_height_ratio_z_score: 0.9799 change in percent body fat for 1-SD lower
    95% CI: [1.3157, 0.6440]
+0.98 (0.64-1.32)


In [121]:
ratio_female_model3 = '-'

## MALE

In [122]:
subset_df = adiposity_df[adiposity_df['Sex']=='Male']

target = subset_df['PBF']

subset_df['MAPF2'] = subset_df['MAPF2'].fillna(0)

#std_devs = subset_df[['BMPHT','leg_length','leg_length_to_height_ratio']].std()

std_devs = subset_df[['BMPHT_z_score','leg_length_z_score','leg_length_to_height_ratio_z_score']].std()

## Height

### Model 1

In [123]:
anthropometric_measures = ['BMPHT_z_score']
model = ols('target ~ BMPHT_z_score+HSAGEIR',data=subset_df).fit()

# Calculate 1-SD lower comparison for each variable in the model
print(f"\n1-SD lower comparison for each variable with confidence intervals in the Basic model:")

# Iterate through each predictor
for predictor in anthropometric_measures:
# Coefficient for the predictor
    coef = model.params[predictor]
    
    # Standard deviation of the predictor
    std_dev = std_devs[predictor] if predictor in std_devs else 0
    
    # Calculate the effect of a 1-SD decrease in the predictor
    BMPHT_male_effect_model1 = coef * (-std_dev)
    
    # Calculate the confidence intervals for the 1-SD lower comparison
    ci_df = model.conf_int()  # Get the confidence interval DataFrame
    
    # Access the lower and upper bounds of the predictor's confidence interval
    if predictor in ci_df.index:
        BMPHT_male_lower_ci_model1 = ci_df.loc[predictor, 0] * (-std_dev)
        BMPHT_male_upper_ci_model1 = ci_df.loc[predictor, 1] * (-std_dev)
        
        # Print the effect and confidence intervals
        print(f"{predictor}: {BMPHT_male_effect_model1:.4f} change in percent body fat for 1-SD lower")
        print(f"    95% CI: [{BMPHT_male_lower_ci_model1:.4f}, {BMPHT_male_upper_ci_model1:.4f}]")
    else:
        print(f"Predictor '{predictor}' not found in the model confidence intervals.")
    
    sign = "+" if BMPHT_male_effect_model1 >= 0 else ""
    height_male_model1 = f"{sign}{abs(BMPHT_male_effect_model1):.2f} ({BMPHT_male_upper_ci_model1:.2f}-{BMPHT_male_lower_ci_model1:.2f})"
    print(height_male_model1)


1-SD lower comparison for each variable with confidence intervals in the Basic model:
BMPHT_z_score: 0.1801 change in percent body fat for 1-SD lower
    95% CI: [0.3802, -0.0201]
+0.18 (-0.02-0.38)


### Model 2

In [124]:
model = ols('target ~ BMPHT_z_score+HSAGEIR+parental_diabetes+physical_activity+education_12_years+income_20000_or_more+smoking_hist+MAPF2',data=subset_df).fit()

# Calculate 1-SD lower comparison for each variable in the model
print(f"\n1-SD lower comparison for each variable with confidence intervals in the Basic model:")

# Iterate through each predictor
for predictor in anthropometric_measures:
# Coefficient for the predictor
    coef = model.params[predictor]
    
    # Standard deviation of the predictor
    std_dev = std_devs[predictor] if predictor in std_devs else 0
    
    # Calculate the effect of a 1-SD decrease in the predictor
    BMPHT_male_effect_model2 = coef * (-std_dev)
    
    # Calculate the confidence intervals for the 1-SD lower comparison
    ci_df = model.conf_int()  # Get the confidence interval DataFrame
    
    # Access the lower and upper bounds of the predictor's confidence interval
    if predictor in ci_df.index:
        BMPHT_male_lower_ci_model2 = ci_df.loc[predictor, 0] * (-std_dev)
        BMPHT_male_upper_ci_model2 = ci_df.loc[predictor, 1] * (-std_dev)
        
        # Print the effect and confidence intervals
        print(f"{predictor}: {BMPHT_male_effect_model2:.4f} change in percent body fat for 1-SD lower")
        print(f"    95% CI: [{BMPHT_male_lower_ci_model2:.4f}, {BMPHT_male_upper_ci_model2:.4f}]")
    else:
        print(f"Predictor '{predictor}' not found in the model confidence intervals.")
    
    sign = "+" if BMPHT_male_effect_model2 >= 0 else ""
    height_male_model2 = f"{sign}{abs(BMPHT_male_effect_model2):.2f} ({BMPHT_male_upper_ci_model2:.2f}-{BMPHT_male_lower_ci_model2:.2f})"
    print(height_male_model2)


1-SD lower comparison for each variable with confidence intervals in the Basic model:
BMPHT_z_score: 0.1472 change in percent body fat for 1-SD lower
    95% CI: [0.3466, -0.0522]
+0.15 (-0.05-0.35)


In [125]:
height_male_model3 = '-'

## Leg length

### Model 1

In [126]:
anthropometric_measures = ['leg_length_z_score']
model = ols('target ~ leg_length_z_score+HSAGEIR',data=subset_df).fit()

# Calculate 1-SD lower comparison for each variable in the model
print(f"\n1-SD lower comparison for each variable with confidence intervals in the Basic model:")

# Iterate through each predictor
for predictor in anthropometric_measures:
# Coefficient for the predictor
    coef = model.params[predictor]
    
    # Standard deviation of the predictor
    std_dev = std_devs[predictor] if predictor in std_devs else 0
    
    # Calculate the effect of a 1-SD decrease in the predictor
    leg_length_male_effect_model1 = coef * (-std_dev)
    
    # Calculate the confidence intervals for the 1-SD lower comparison
    ci_df = model.conf_int()  # Get the confidence interval DataFrame
    
    # Access the lower and upper bounds of the predictor's confidence interval
    if predictor in ci_df.index:
        leg_length_male_lower_ci_model1 = ci_df.loc[predictor, 0] * (-std_dev)
        leg_length_male_upper_ci_model1 = ci_df.loc[predictor, 1] * (-std_dev)
        
        # Print the effect and confidence intervals
        print(f"{predictor}: {BMPHT_male_effect_model1:.4f} change in percent body fat for 1-SD lower")
        print(f"    95% CI: [{BMPHT_male_lower_ci_model1:.4f}, {BMPHT_male_upper_ci_model1:.4f}]")
    else:
        print(f"Predictor '{predictor}' not found in the model confidence intervals.")
    
    sign = "+" if leg_length_male_effect_model1 >= 0 else ""
    leg_length_male_model1 = f"{sign}{abs(leg_length_male_effect_model1):.2f} ({leg_length_male_upper_ci_model1:.2f}-{leg_length_male_lower_ci_model1:.2f})"
    print(leg_length_male_model1)


1-SD lower comparison for each variable with confidence intervals in the Basic model:
leg_length_z_score: 0.1801 change in percent body fat for 1-SD lower
    95% CI: [0.3802, -0.0201]
+0.17 (-0.03-0.37)


### Model 2

In [127]:
model = ols('target ~ leg_length_z_score+HSAGEIR+parental_diabetes+physical_activity+education_12_years+income_20000_or_more+smoking_hist+MAPF2',data=subset_df).fit()

# Calculate 1-SD lower comparison for each variable in the model
print(f"\n1-SD lower comparison for each variable with confidence intervals in the Basic model:")

# Iterate through each predictor
for predictor in anthropometric_measures:
# Coefficient for the predictor
    coef = model.params[predictor]
    
    # Standard deviation of the predictor
    std_dev = std_devs[predictor] if predictor in std_devs else 0
    
    # Calculate the effect of a 1-SD decrease in the predictor
    leg_length_male_effect_model2 = coef * (-std_dev)
    
    # Calculate the confidence intervals for the 1-SD lower comparison
    ci_df = model.conf_int()  # Get the confidence interval DataFrame
    
    # Access the lower and upper bounds of the predictor's confidence interval
    if predictor in ci_df.index:
        leg_length_male_lower_ci_model2 = ci_df.loc[predictor, 0] * (-std_dev)
        leg_length_male_upper_ci_model2 = ci_df.loc[predictor, 1] * (-std_dev)
        
        # Print the effect and confidence intervals
        print(f"{predictor}: {leg_length_male_effect_model2:.4f} change in percent body fat for 1-SD lower")
        print(f"    95% CI: [{leg_length_male_lower_ci_model2:.4f}, {leg_length_male_upper_ci_model2:.4f}]")
    else:
        print(f"Predictor '{predictor}' not found in the model confidence intervals.")
    
    sign = "+" if leg_length_male_effect_model2 >= 0 else ""
    leg_length_male_model2 = f"{sign}{abs(leg_length_male_effect_model2):.2f} ({leg_length_male_upper_ci_model2:.2f}-{leg_length_male_lower_ci_model2:.2f})"
    print(leg_length_male_model2)


1-SD lower comparison for each variable with confidence intervals in the Basic model:
leg_length_z_score: 0.1279 change in percent body fat for 1-SD lower
    95% CI: [0.3255, -0.0697]
+0.13 (-0.07-0.33)


In [128]:
leg_length_male_model3 = '-'

## Leg length to height ratio

### Model 1

In [129]:
anthropometric_measures = ['leg_length_to_height_ratio_z_score']
model = ols('target ~ leg_length_to_height_ratio_z_score+HSAGEIR',data=subset_df).fit()

# Calculate 1-SD lower comparison for each variable in the model
print(f"\n1-SD lower comparison for each variable with confidence intervals in the Basic model:")

# Iterate through each predictor
for predictor in anthropometric_measures:
# Coefficient for the predictor
    coef = model.params[predictor]
    
    # Standard deviation of the predictor
    std_dev = std_devs[predictor] if predictor in std_devs else 0
    
    # Calculate the effect of a 1-SD decrease in the predictor
    ratio_male_effect_model1 = coef * (-std_dev)
    
    # Calculate the confidence intervals for the 1-SD lower comparison
    ci_df = model.conf_int()  # Get the confidence interval DataFrame
    
    # Access the lower and upper bounds of the predictor's confidence interval
    if predictor in ci_df.index:
        ratio_male_lower_ci_model1 = ci_df.loc[predictor, 0] * (-std_dev)
        ratio_male_upper_ci_model1 = ci_df.loc[predictor, 1] * (-std_dev)
        
        # Print the effect and confidence intervals
        print(f"{predictor}: {ratio_male_effect_model1:.4f} change in percent body fat for 1-SD lower")
        print(f"    95% CI: [{ratio_male_lower_ci_model1:.4f}, {ratio_male_upper_ci_model1:.4f}]")
    else:
        print(f"Predictor '{predictor}' not found in the model confidence intervals.")
    
    sign = "+" if ratio_male_effect_model1 >= 0 else ""
    ratio_male_model1 = f"{sign}{abs(ratio_male_effect_model1):.2f} ({ratio_male_upper_ci_model1:.2f}-{ratio_male_lower_ci_model1:.2f})"
    print(ratio_male_model1)


1-SD lower comparison for each variable with confidence intervals in the Basic model:
leg_length_to_height_ratio_z_score: 0.2383 change in percent body fat for 1-SD lower
    95% CI: [0.4388, 0.0378]
+0.24 (0.04-0.44)


### Model 2

In [130]:
model = ols('target ~ leg_length_to_height_ratio_z_score+HSAGEIR+parental_diabetes+physical_activity+education_12_years+income_20000_or_more+smoking_hist+MAPF2',data=subset_df).fit()

# Calculate 1-SD lower comparison for each variable in the model
print(f"\n1-SD lower comparison for each variable with confidence intervals in the Basic model:")

# Iterate through each predictor
for predictor in anthropometric_measures:
# Coefficient for the predictor
    coef = model.params[predictor]
    
    # Standard deviation of the predictor
    std_dev = std_devs[predictor] if predictor in std_devs else 0
    
    # Calculate the effect of a 1-SD decrease in the predictor
    ratio_male_effect_model2 = coef * (-std_dev)
    
    # Calculate the confidence intervals for the 1-SD lower comparison
    ci_df = model.conf_int()  # Get the confidence interval DataFrame
    
    # Access the lower and upper bounds of the predictor's confidence interval
    if predictor in ci_df.index:
        ratio_male_lower_ci_model2 = ci_df.loc[predictor, 0] * (-std_dev)
        ratio_male_upper_ci_model2 = ci_df.loc[predictor, 1] * (-std_dev)
        
        # Print the effect and confidence intervals
        print(f"{predictor}: {ratio_male_effect_model2:.4f} change in percent body fat for 1-SD lower")
        print(f"    95% CI: [{ratio_male_lower_ci_model2:.4f}, {ratio_male_upper_ci_model2:.4f}]")
    else:
        print(f"Predictor '{predictor}' not found in the model confidence intervals.")
    
    sign = "+" if ratio_male_effect_model2 >= 0 else ""
    ratio_male_model2 = f"{sign}{abs(ratio_male_effect_model2):.2f} ({ratio_male_upper_ci_model2:.2f}-{ratio_male_lower_ci_model2:.2f})"
    print(ratio_male_model2)


1-SD lower comparison for each variable with confidence intervals in the Basic model:
leg_length_to_height_ratio_z_score: 0.2018 change in percent body fat for 1-SD lower
    95% CI: [0.4012, 0.0024]
+0.20 (0.00-0.40)


In [131]:
ratio_male_model3 = '-'

# HOMA-IR

In [132]:
HOMA_df['HOMA_IR_log'] = np.log(HOMA_df['HOMA2_IR'])

HOMA_df['MAPF2'] = HOMA_df['MAPF2'].fillna(0)

std_devs = HOMA_df[['BMPHT_z_score','leg_length_z_score','leg_length_to_height_ratio_z_score']].std()

## Height

### Model 1

In [133]:
anthropometric_measures = ['BMPHT_z_score']
model = ols('HOMA_IR_log ~ BMPHT_z_score+HSAGEIR',data=HOMA_df).fit()

# Calculate 1-SD lower comparison for each variable in the model
print(f"\n1-SD lower comparison for each variable with confidence intervals in the Basic model:")

# Iterate through each predictor
for predictor in anthropometric_measures:
# Coefficient for the predictor
    coef = model.params[predictor]
    
    # Standard deviation of the predictor
    std_dev = std_devs[predictor] if predictor in std_devs else 0
    
    # Calculate the effect of a 1-SD decrease in the predictor
    BMPHT_HOMA_effect_model1 = np.exp(coef * (-std_dev))
    
    # Calculate the confidence intervals for the 1-SD lower comparison
    ci_df = model.conf_int()  # Get the confidence interval DataFrame
    
    # Access the lower and upper bounds of the predictor's confidence interval
    if predictor in ci_df.index:
        BMPHT_HOMA_lower_ci_model1 = np.exp(ci_df.loc[predictor, 0] * (-std_dev))
        BMPHT_HOMA_upper_ci_model1 = np.exp(ci_df.loc[predictor, 1] * (-std_dev))
        
        # Print the effect and confidence intervals
        print(f"{predictor}: {BMPHT_HOMA_effect_model1:.2f} change in percent body fat for 1-SD lower")
        print(f"    95% CI: [{BMPHT_HOMA_lower_ci_model1:.2f}, {BMPHT_HOMA_upper_ci_model1:.2f}]")
    else:
        print(f"Predictor '{predictor}' not found in the model confidence intervals.")
    
    sign = "+" if BMPHT_HOMA_effect_model1 >= 0 else ""
    height_HOMA_model1 = f"{abs(BMPHT_HOMA_effect_model1):.2f} ({BMPHT_HOMA_upper_ci_model1:.2f}-{BMPHT_HOMA_lower_ci_model1:.2f})"
    print(height_HOMA_model1)


1-SD lower comparison for each variable with confidence intervals in the Basic model:
BMPHT_z_score: 1.00 change in percent body fat for 1-SD lower
    95% CI: [1.01, 0.98]
1.00 (0.98-1.01)


### Model 2

In [134]:
model = ols('HOMA_IR_log ~ BMPHT_z_score+HSAGEIR+parental_diabetes+physical_activity+education_12_years+income_20000_or_more+smoking_hist+MAPF2',data=HOMA_df).fit()

# Calculate 1-SD lower comparison for each variable in the model
print(f"\n1-SD lower comparison for each variable with confidence intervals in the Basic model:")

# Iterate through each predictor
for predictor in anthropometric_measures:
# Coefficient for the predictor
    coef = model.params[predictor]
    
    # Standard deviation of the predictor
    std_dev = std_devs[predictor] if predictor in std_devs else 0
    
    # Calculate the effect of a 1-SD decrease in the predictor
    BMPHT_HOMA_effect_model2 = np.exp(coef * (-std_dev))
    
    # Calculate the confidence intervals for the 1-SD lower comparison
    ci_df = model.conf_int()  # Get the confidence interval DataFrame
    
    # Access the lower and upper bounds of the predictor's confidence interval
    if predictor in ci_df.index:
        BMPHT_HOMA_lower_ci_model2 = np.exp(ci_df.loc[predictor, 0] * (-std_dev))
        BMPHT_HOMA_upper_ci_model2 = np.exp(ci_df.loc[predictor, 1] * (-std_dev))
        
        # Print the effect and confidence intervals
        print(f"{predictor}: {BMPHT_HOMA_effect_model2:.2f} change in percent body fat for 1-SD lower")
        print(f"    95% CI: [{BMPHT_HOMA_lower_ci_model2:.2f}, {BMPHT_HOMA_upper_ci_model2:.2f}]")
    else:
        print(f"Predictor '{predictor}' not found in the model confidence intervals.")

    sign = "+" if BMPHT_HOMA_effect_model2 >= 0 else ""
    height_HOMA_model2 = f"{abs(BMPHT_HOMA_effect_model2):.2f} ({BMPHT_HOMA_upper_ci_model2:.2f}-{BMPHT_HOMA_lower_ci_model2:.2f})"
    print(height_HOMA_model2)


1-SD lower comparison for each variable with confidence intervals in the Basic model:
BMPHT_z_score: 0.99 change in percent body fat for 1-SD lower
    95% CI: [1.01, 0.98]
0.99 (0.98-1.01)


### Model 3

In [135]:
model = ols('HOMA_IR_log ~ BMPHT_z_score+HSAGEIR+parental_diabetes+physical_activity+education_12_years+income_20000_or_more+smoking_hist+MAPF2+PBF+PBF*Sex',data=HOMA_df).fit()

# Calculate 1-SD lower comparison for each variable in the model
print(f"\n1-SD lower comparison for each variable with confidence intervals in the Basic model:")

# Iterate through each predictor
for predictor in anthropometric_measures:
# Coefficient for the predictor
    coef = model.params[predictor]
    
    # Standard deviation of the predictor
    std_dev = std_devs[predictor] if predictor in std_devs else 0
    
    # Calculate the effect of a 1-SD decrease in the predictor
    BMPHT_HOMA_effect_model3 = np.exp(coef * (-std_dev))
    
    # Calculate the confidence intervals for the 1-SD lower comparison
    ci_df = model.conf_int()  # Get the confidence interval DataFrame
    
    # Access the lower and upper bounds of the predictor's confidence interval
    if predictor in ci_df.index:
        BMPHT_HOMA_lower_ci_model3 = np.exp(ci_df.loc[predictor, 0] * (-std_dev))
        BMPHT_HOMA_upper_ci_model3 = np.exp(ci_df.loc[predictor, 1] * (-std_dev))
        
        # Print the effect and confidence intervals
        print(f"{predictor}: {BMPHT_HOMA_effect_model3:.2f} change in percent body fat for 1-SD lower")
        print(f"    95% CI: [{BMPHT_HOMA_lower_ci_model3:.2f}, {BMPHT_HOMA_upper_ci_model3:.2f}]")
    else:
        print(f"Predictor '{predictor}' not found in the model confidence intervals.")
    
    sign = "+" if BMPHT_HOMA_effect_model3 >= 0 else ""
    height_HOMA_model3 = f"{abs(BMPHT_HOMA_effect_model3):.2f} ({BMPHT_HOMA_upper_ci_model3:.2f}-{BMPHT_HOMA_lower_ci_model3:.2f})"
    print(height_HOMA_model3)


1-SD lower comparison for each variable with confidence intervals in the Basic model:
BMPHT_z_score: 0.98 change in percent body fat for 1-SD lower
    95% CI: [0.99, 0.97]
0.98 (0.97-0.99)


## Leg Length

### Model 1

In [136]:
anthropometric_measures = ['leg_length_z_score']
model = ols('HOMA_IR_log ~ leg_length_z_score+HSAGEIR',data=HOMA_df).fit()

# Calculate 1-SD lower comparison for each variable in the model
print(f"\n1-SD lower comparison for each variable with confidence intervals in the Basic model:")

# Iterate through each predictor
for predictor in anthropometric_measures:
# Coefficient for the predictor
    coef = model.params[predictor]
    
    # Standard deviation of the predictor
    std_dev = std_devs[predictor] if predictor in std_devs else 0
    
    # Calculate the effect of a 1-SD decrease in the predictor
    leg_length_HOMA_effect_model1 = np.exp(coef * (-std_dev))
    
    # Calculate the confidence intervals for the 1-SD lower comparison
    ci_df = model.conf_int()  # Get the confidence interval DataFrame
    
    # Access the lower and upper bounds of the predictor's confidence interval
    if predictor in ci_df.index:
        leg_length_HOMA_lower_ci_model1 = np.exp(ci_df.loc[predictor, 0] * (-std_dev))
        leg_length_HOMA_upper_ci_model1 = np.exp(ci_df.loc[predictor, 1] * (-std_dev))
        
        # Print the effect and confidence intervals
        print(f"{predictor}: {leg_length_HOMA_effect_model1:.2f} change in percent body fat for 1-SD lower")
        print(f"    95% CI: [{leg_length_HOMA_lower_ci_model1:.2f}, {leg_length_HOMA_upper_ci_model1:.2f}]")
    else:
        print(f"Predictor '{predictor}' not found in the model confidence intervals.")
    
    sign = "+" if leg_length_HOMA_effect_model1 >= 0 else ""
    leg_length_HOMA_model1 = f"{abs(leg_length_HOMA_effect_model1):.2f} ({leg_length_HOMA_upper_ci_model1:.2f}-{leg_length_HOMA_lower_ci_model1:.2f})"
    print(leg_length_HOMA_model1)


1-SD lower comparison for each variable with confidence intervals in the Basic model:
leg_length_z_score: 1.02 change in percent body fat for 1-SD lower
    95% CI: [1.04, 1.01]
1.02 (1.01-1.04)


### Model 2

In [137]:
model = ols('HOMA_IR_log ~ leg_length_z_score+HSAGEIR+parental_diabetes+physical_activity+education_12_years+income_20000_or_more+smoking_hist+MAPF2',data=HOMA_df).fit()

# Calculate 1-SD lower comparison for each variable in the model
print(f"\n1-SD lower comparison for each variable with confidence intervals in the Basic model:")

# Iterate through each predictor
for predictor in anthropometric_measures:
# Coefficient for the predictor
    coef = model.params[predictor]
    
    # Standard deviation of the predictor
    std_dev = std_devs[predictor] if predictor in std_devs else 0
    
    # Calculate the effect of a 1-SD decrease in the predictor
    leg_length_HOMA_effect_model2 = np.exp(coef * (-std_dev))
    
    # Calculate the confidence intervals for the 1-SD lower comparison
    ci_df = model.conf_int()  # Get the confidence interval DataFrame
    
    # Access the lower and upper bounds of the predictor's confidence interval
    if predictor in ci_df.index:
        leg_length_HOMA_lower_ci_model2 = np.exp(ci_df.loc[predictor, 0] * (-std_dev))
        leg_length_HOMA_upper_ci_model2 = np.exp(ci_df.loc[predictor, 1] * (-std_dev))
        
        # Print the effect and confidence intervals
        print(f"{predictor}: {leg_length_HOMA_effect_model2:.2f} change in percent body fat for 1-SD lower")
        print(f"    95% CI: [{leg_length_HOMA_lower_ci_model2:.2f}, {leg_length_HOMA_upper_ci_model2:.2f}]")
    else:
        print(f"Predictor '{predictor}' not found in the model confidence intervals.")

    sign = "+" if leg_length_HOMA_effect_model2 >= 0 else ""
    leg_length_HOMA_model2 = f"{abs(leg_length_HOMA_effect_model2):.2f} ({leg_length_HOMA_upper_ci_model2:.2f}-{leg_length_HOMA_lower_ci_model2:.2f})"
    print(leg_length_HOMA_model2)


1-SD lower comparison for each variable with confidence intervals in the Basic model:
leg_length_z_score: 1.02 change in percent body fat for 1-SD lower
    95% CI: [1.03, 1.00]
1.02 (1.00-1.03)


### Model 3

In [138]:
model = ols('HOMA_IR_log ~ leg_length_z_score+HSAGEIR+parental_diabetes+physical_activity+education_12_years+income_20000_or_more+smoking_hist+MAPF2+PBF+PBF*Sex',data=HOMA_df).fit()

# Calculate 1-SD lower comparison for each variable in the model
print(f"\n1-SD lower comparison for each variable with confidence intervals in the Basic model:")

# Iterate through each predictor
for predictor in anthropometric_measures:
# Coefficient for the predictor
    coef = model.params[predictor]
    
    # Standard deviation of the predictor
    std_dev = std_devs[predictor] if predictor in std_devs else 0
    
    # Calculate the effect of a 1-SD decrease in the predictor
    leg_length_HOMA_effect_model3 = np.exp(coef * (-std_dev))
    
    # Calculate the confidence intervals for the 1-SD lower comparison
    ci_df = model.conf_int()  # Get the confidence interval DataFrame
    
    # Access the lower and upper bounds of the predictor's confidence interval
    if predictor in ci_df.index:
        leg_length_HOMA_lower_ci_model3 = np.exp(ci_df.loc[predictor, 0] * (-std_dev))
        leg_length_HOMA_upper_ci_model3 = np.exp(ci_df.loc[predictor, 1] * (-std_dev))
        
        # Print the effect and confidence intervals
        print(f"{predictor}: {leg_length_HOMA_effect_model3:.2f} change in percent body fat for 1-SD lower")
        print(f"    95% CI: [{leg_length_HOMA_lower_ci_model3:.2f}, {leg_length_HOMA_upper_ci_model3:.2f}]")
    else:
        print(f"Predictor '{predictor}' not found in the model confidence intervals.")
    
    sign = "+" if leg_length_HOMA_effect_model3 >= 0 else ""
    leg_length_HOMA_model3 = f"{abs(leg_length_HOMA_effect_model3):.2f} ({leg_length_HOMA_upper_ci_model3:.2f}-{leg_length_HOMA_lower_ci_model3:.2f})"
    print(leg_length_HOMA_model3)


1-SD lower comparison for each variable with confidence intervals in the Basic model:
leg_length_z_score: 1.00 change in percent body fat for 1-SD lower
    95% CI: [1.01, 0.99]
1.00 (0.99-1.01)


## Leg Length to Height ratio

### Model 1

In [139]:
anthropometric_measures = ['leg_length_to_height_ratio_z_score']
model = ols('HOMA_IR_log ~ leg_length_to_height_ratio_z_score+HSAGEIR',data=HOMA_df).fit()

# Calculate 1-SD lower comparison for each variable in the model
print(f"\n1-SD lower comparison for each variable with confidence intervals in the Basic model:")

# Iterate through each predictor
for predictor in anthropometric_measures:
# Coefficient for the predictor
    coef = model.params[predictor]
    
    # Standard deviation of the predictor
    std_dev = std_devs[predictor] if predictor in std_devs else 0
    
    # Calculate the effect of a 1-SD decrease in the predictor
    leg_length_to_height_ratio_HOMA_effect_model1 = np.exp(coef * (-std_dev))
    
    # Calculate the confidence intervals for the 1-SD lower comparison
    ci_df = model.conf_int()  # Get the confidence interval DataFrame
    
    # Access the lower and upper bounds of the predictor's confidence interval
    if predictor in ci_df.index:
        leg_length_to_height_ratio_HOMA_lower_ci_model1 = np.exp(ci_df.loc[predictor, 0] * (-std_dev))
        leg_length_to_height_ratio_HOMA_upper_ci_model1 = np.exp(ci_df.loc[predictor, 1] * (-std_dev))
        
        # Print the effect and confidence intervals
        print(f"{predictor}: {leg_length_to_height_ratio_HOMA_effect_model1:.2f} change in percent body fat for 1-SD lower")
        print(f"    95% CI: [{leg_length_to_height_ratio_HOMA_lower_ci_model1:.2f}, {leg_length_to_height_ratio_HOMA_upper_ci_model1:.2f}]")
    else:
        print(f"Predictor '{predictor}' not found in the model confidence intervals.")
    
    sign = "+" if leg_length_to_height_ratio_HOMA_effect_model1 >= 0 else ""
    ratio_HOMA_model1 = f"{abs(leg_length_to_height_ratio_HOMA_effect_model1):.2f} ({leg_length_to_height_ratio_HOMA_upper_ci_model1:.2f}-{leg_length_to_height_ratio_HOMA_lower_ci_model1:.2f})"
    print(ratio_HOMA_model1)


1-SD lower comparison for each variable with confidence intervals in the Basic model:
leg_length_to_height_ratio_z_score: 1.06 change in percent body fat for 1-SD lower
    95% CI: [1.07, 1.04]
1.06 (1.04-1.07)


### Model 2

In [140]:
model = ols('HOMA_IR_log ~ leg_length_to_height_ratio_z_score+HSAGEIR+parental_diabetes+physical_activity+education_12_years+income_20000_or_more+smoking_hist+MAPF2',data=HOMA_df).fit()

# Calculate 1-SD lower comparison for each variable in the model
print(f"\n1-SD lower comparison for each variable with confidence intervals in the Basic model:")

# Iterate through each predictor
for predictor in anthropometric_measures:
# Coefficient for the predictor
    coef = model.params[predictor]
    
    # Standard deviation of the predictor
    std_dev = std_devs[predictor] if predictor in std_devs else 0
    
    # Calculate the effect of a 1-SD decrease in the predictor
    leg_length_to_height_ratio_HOMA_effect_model2 = np.exp(coef * (-std_dev))
    
    # Calculate the confidence intervals for the 1-SD lower comparison
    ci_df = model.conf_int()  # Get the confidence interval DataFrame
    
    # Access the lower and upper bounds of the predictor's confidence interval
    if predictor in ci_df.index:
        leg_length_to_height_ratio_HOMA_lower_ci_model2 = np.exp(ci_df.loc[predictor, 0] * (-std_dev))
        leg_length_to_height_ratio_HOMA_upper_ci_model2 = np.exp(ci_df.loc[predictor, 1] * (-std_dev))
        
        # Print the effect and confidence intervals
        print(f"{predictor}: {leg_length_to_height_ratio_HOMA_effect_model2:.2f} change in percent body fat for 1-SD lower")
        print(f"    95% CI: [{leg_length_to_height_ratio_HOMA_lower_ci_model2:.2f}, {leg_length_to_height_ratio_HOMA_upper_ci_model2:.2f}]")
    else:
        print(f"Predictor '{predictor}' not found in the model confidence intervals.")
    
    sign = "+" if leg_length_to_height_ratio_HOMA_effect_model2 >= 0 else ""
    ratio_HOMA_model2 = f"{abs(leg_length_to_height_ratio_HOMA_effect_model2):.2f} ({leg_length_to_height_ratio_HOMA_upper_ci_model2:.2f}-{leg_length_to_height_ratio_HOMA_lower_ci_model2:.2f})"
    print(ratio_HOMA_model2)


1-SD lower comparison for each variable with confidence intervals in the Basic model:
leg_length_to_height_ratio_z_score: 1.05 change in percent body fat for 1-SD lower
    95% CI: [1.07, 1.04]
1.05 (1.04-1.07)


### Model 3

In [141]:
model = ols('HOMA_IR_log ~ leg_length_to_height_ratio_z_score+HSAGEIR+parental_diabetes+physical_activity+education_12_years+income_20000_or_more+smoking_hist+MAPF2+PBF+PBF*Sex',data=HOMA_df).fit()

# Calculate 1-SD lower comparison for each variable in the model
print(f"\n1-SD lower comparison for each variable with confidence intervals in the Basic model:")

# Iterate through each predictor
for predictor in anthropometric_measures:
# Coefficient for the predictor
    coef = model.params[predictor]
    
    # Standard deviation of the predictor
    std_dev = std_devs[predictor] if predictor in std_devs else 0
    
    # Calculate the effect of a 1-SD decrease in the predictor
    leg_length_to_height_ratio_HOMA_effect_model3 = np.exp(coef * (-std_dev))
    
    # Calculate the confidence intervals for the 1-SD lower comparison
    ci_df = model.conf_int()  # Get the confidence interval DataFrame
    
    # Access the lower and upper bounds of the predictor's confidence interval
    if predictor in ci_df.index:
        leg_length_to_height_ratio_HOMA_lower_ci_model3 = np.exp(ci_df.loc[predictor, 0] * (-std_dev))
        leg_length_to_height_ratio_HOMA_upper_ci_model3 = np.exp(ci_df.loc[predictor, 1] * (-std_dev))
        
        # Print the effect and confidence intervals
        print(f"{predictor}: {leg_length_to_height_ratio_HOMA_effect_model3:.2f} change in percent body fat for 1-SD lower")
        print(f"    95% CI: [{leg_length_to_height_ratio_HOMA_lower_ci_model3:.2f}, {leg_length_to_height_ratio_HOMA_upper_ci_model3:.2f}]")
    else:
        print(f"Predictor '{predictor}' not found in the model confidence intervals.")
    
    sign = "+" if leg_length_to_height_ratio_HOMA_effect_model3 >= 0 else ""
    ratio_HOMA_model3 = f"{abs(leg_length_to_height_ratio_HOMA_effect_model3):.2f} ({leg_length_to_height_ratio_HOMA_upper_ci_model3:.2f}-{leg_length_to_height_ratio_HOMA_lower_ci_model3:.2f})"
    print(ratio_HOMA_model3)


1-SD lower comparison for each variable with confidence intervals in the Basic model:
leg_length_to_height_ratio_z_score: 1.04 change in percent body fat for 1-SD lower
    95% CI: [1.05, 1.02]
1.04 (1.02-1.05)


# Prevalence Ratio

In [142]:
ogtt_df['GT_Category_mapped'] = ogtt_df['GT_Category'].map({
    'normal': 1 ,
    'IGT': 2,
    'diabetes': 3
})
ogtt_df['GT_Category_mapped'] = ogtt_df['GT_Category_mapped'].astype(int)

ogtt_df['MAPF2'] = ogtt_df['MAPF2'].fillna(0)

ogtt_df = pd.merge(ogtt_df, adiposity_df[['SEQN','PBF']], on='SEQN', how='left')

std_devs = ogtt_df[['BMPHT_z_score','leg_length_z_score','leg_length_to_height_ratio_z_score']].std()

medians = ogtt_df['PBF'].median()

# Impute missing values with the median
ogtt_df['PBF'] = ogtt_df['PBF'].fillna(medians)

## Height

In [143]:
std_dev = std_devs['BMPHT_z_score']
anthropometric_measures = [['BMPHT_z_score',0],['BMPHT_z_score',1]]

### Model 1

In [144]:
model = mnlogit('GT_Category_mapped ~ BMPHT_z_score+HSAGEIR',data=ogtt_df).fit()

Optimization terminated successfully.
         Current function value: 0.987781
         Iterations 5


### IGT

In [145]:
coef = model.params[0]['BMPHT_z_score']
BMPHT_IGT_Model1_effect = np.exp(coef * (-std_dev))

ci_df = model.conf_int().reset_index()

BMPHT_IGT_Model1_upper = np.exp(ci_df.loc[(ci_df['GT_Category_mapped'] == '2') & (ci_df['level_1'] == 'BMPHT_z_score'), 'upper'].values[0] * (-std_dev))
BMPHT_IGT_Model1_lower = np.exp(ci_df.loc[(ci_df['GT_Category_mapped'] == '2') & (ci_df['level_1'] == 'BMPHT_z_score'), 'lower'].values[0] * (-std_dev))

height_IGT_model1 = f"{abs(BMPHT_IGT_Model1_effect):.2f} ({BMPHT_IGT_Model1_upper:.2f}-{BMPHT_IGT_Model1_lower:.2f})"
print(height_IGT_model1)


1.12 (1.05-1.19)


### Diabetes

In [146]:
coef = model.params[1]['BMPHT_z_score']
BMPHT_Diabetes_Model1_effect = np.exp(coef*(-std_dev))

ci_df = model.conf_int().reset_index()

BMPHT_Diabetes_Model1_upper = np.exp(ci_df.loc[(ci_df['GT_Category_mapped'] == '3') & (ci_df['level_1'] == 'BMPHT_z_score'), 'upper'].values[0] * (-std_dev))
BMPHT_Diabetes_Model1_lower = np.exp(ci_df.loc[(ci_df['GT_Category_mapped'] == '3') & (ci_df['level_1'] == 'BMPHT_z_score'), 'lower'].values[0] * (-std_dev))

height_diabetes_model1 = f"{abs(BMPHT_Diabetes_Model1_effect):.2f} ({BMPHT_Diabetes_Model1_upper:.2f}-{BMPHT_Diabetes_Model1_lower:.2f})"
print(height_diabetes_model1)

1.10 (1.03-1.18)


### Model 2

In [147]:
model = mnlogit('GT_Category_mapped ~ BMPHT_z_score+HSAGEIR+parental_diabetes+physical_activity+education_12_years+income_20000_or_more+smoking_hist+MAPF2',data=ogtt_df).fit()

Optimization terminated successfully.
         Current function value: 0.955146
         Iterations 6


### IGT

In [148]:
coef = model.params[0]['BMPHT_z_score']
BMPHT_IGT_Model2_effect = np.exp(coef*(-std_dev))

ci_df = model.conf_int().reset_index()

BMPHT_IGT_Model2_upper = np.exp(ci_df.loc[(ci_df['GT_Category_mapped'] == '2') & (ci_df['level_1'] == 'BMPHT_z_score'), 'upper'].values[0] * (-std_dev))
BMPHT_IGT_Model2_lower = np.exp(ci_df.loc[(ci_df['GT_Category_mapped'] == '2') & (ci_df['level_1'] == 'BMPHT_z_score'), 'lower'].values[0] * (-std_dev))

height_IGT_model2 = f"{abs(BMPHT_IGT_Model2_effect):.2f} ({BMPHT_IGT_Model2_upper:.2f}-{BMPHT_IGT_Model2_lower:.2f})"
print(height_IGT_model2)

1.10 (1.03-1.17)


### Diabetes

In [149]:
coef = model.params[1]['BMPHT_z_score']
BMPHT_Diabetes_Model2_effect = np.exp(coef*(-std_dev))

ci_df = model.conf_int().reset_index()

BMPHT_Diabetes_Model2_upper = np.exp(ci_df.loc[(ci_df['GT_Category_mapped'] == '3') & (ci_df['level_1'] == 'BMPHT_z_score'), 'upper'].values[0] * (-std_dev))
BMPHT_Diabetes_Model2_lower = np.exp(ci_df.loc[(ci_df['GT_Category_mapped'] == '3') & (ci_df['level_1'] == 'BMPHT_z_score'), 'lower'].values[0] * (-std_dev))

height_diabetes_model2 = f"{abs(BMPHT_Diabetes_Model2_effect):.2f} ({BMPHT_Diabetes_Model2_upper:.2f}-{BMPHT_Diabetes_Model2_lower:.2f})"
print(height_diabetes_model2)

1.06 (0.98-1.14)


## Model 3

In [150]:
model = mnlogit('GT_Category_mapped ~ BMPHT_z_score+HSAGEIR+parental_diabetes+physical_activity+education_12_years+income_20000_or_more+smoking_hist+MAPF2+PBF+PBF*Sex',data=ogtt_df).fit()

Optimization terminated successfully.
         Current function value: 0.946736
         Iterations 6


### IGT

In [151]:
coef = model.params[0]['BMPHT_z_score']
BMPHT_IGT_Model3_effect = np.exp(coef*(-std_dev))

ci_df = model.conf_int().reset_index()

BMPHT_IGT_Model3_upper = np.exp(ci_df.loc[(ci_df['GT_Category_mapped'] == '2') & (ci_df['level_1'] == 'BMPHT_z_score'), 'upper'].values[0] * (-std_dev))
BMPHT_IGT_Model3_lower = np.exp(ci_df.loc[(ci_df['GT_Category_mapped'] == '2') & (ci_df['level_1'] == 'BMPHT_z_score'), 'lower'].values[0] * (-std_dev))

height_IGT_model3 = f"{abs(BMPHT_IGT_Model3_effect):.2f} ({BMPHT_IGT_Model3_upper:.2f}-{BMPHT_IGT_Model3_lower:.2f})"
print(height_IGT_model3)

1.09 (1.02-1.16)


### Diabetes

In [152]:
coef = model.params[1]['BMPHT_z_score']
BMPHT_Diabetes_Model3_effect = np.exp(coef*(-std_dev))

ci_df = model.conf_int().reset_index()

BMPHT_Diabetes_Model3_upper = np.exp(ci_df.loc[(ci_df['GT_Category_mapped'] == '3') & (ci_df['level_1'] == 'BMPHT_z_score'), 'upper'].values[0] * (-std_dev))
BMPHT_Diabetes_Model3_lower = np.exp(ci_df.loc[(ci_df['GT_Category_mapped'] == '3') & (ci_df['level_1'] == 'BMPHT_z_score'), 'lower'].values[0] * (-std_dev))

height_diabetes_model3 = f"{abs(BMPHT_Diabetes_Model3_effect):.2f} ({BMPHT_Diabetes_Model3_upper:.2f}-{BMPHT_Diabetes_Model3_lower:.2f})"
print(height_diabetes_model3)

1.05 (0.97-1.13)


## Leg Length

In [153]:
std_dev = std_devs['leg_length_z_score']
anthropometric_measures = [['leg_length_z_score',0],['leg_length_z_score',1]]

## Model 1

In [154]:
model = mnlogit('GT_Category_mapped ~ leg_length_z_score+HSAGEIR',data=ogtt_df).fit()

Optimization terminated successfully.
         Current function value: 0.987197
         Iterations 5


### IGT

In [155]:
coef = model.params[0]['leg_length_z_score']
leg_length_IGT_Model1_effect = np.exp(coef*(-std_dev))

ci_df = model.conf_int().reset_index()

leg_length_IGT_Model1_upper = np.exp(ci_df.loc[(ci_df['GT_Category_mapped'] == '2') & (ci_df['level_1'] == 'leg_length_z_score'), 'upper'].values[0] * (-std_dev))
leg_length_IGT_Model1_lower = np.exp(ci_df.loc[(ci_df['GT_Category_mapped'] == '2') & (ci_df['level_1'] == 'leg_length_z_score'), 'lower'].values[0] * (-std_dev))

leg_length_IGT_model1 = f"{abs(leg_length_IGT_Model1_effect):.2f} ({leg_length_IGT_Model1_upper:.2f}-{leg_length_IGT_Model1_lower:.2f})"
print(leg_length_IGT_model1)

1.10 (1.03-1.17)


### Diabetes

In [156]:
coef = model.params[1]['leg_length_z_score']
leg_length_Diabetes_Model1_effect = np.exp(coef*(-std_dev))

ci_df = model.conf_int().reset_index()

leg_length_Diabetes_Model1_upper = np.exp(ci_df.loc[(ci_df['GT_Category_mapped'] == '3') & (ci_df['level_1'] == 'leg_length_z_score'), 'upper'].values[0] * (-std_dev))
leg_length_Diabetes_Model1_lower = np.exp(ci_df.loc[(ci_df['GT_Category_mapped'] == '3') & (ci_df['level_1'] == 'leg_length_z_score'), 'lower'].values[0] * (-std_dev))

leg_length_diabetes_model1 = f"{abs(leg_length_Diabetes_Model1_effect):.2f} ({leg_length_Diabetes_Model1_upper:.2f}-{leg_length_Diabetes_Model1_lower:.2f})"
print(leg_length_diabetes_model1)

1.17 (1.09-1.25)


## Model 2

In [157]:
model = mnlogit('GT_Category_mapped ~ leg_length_z_score+HSAGEIR+parental_diabetes+physical_activity+education_12_years+income_20000_or_more+smoking_hist+MAPF2',data=ogtt_df).fit()

Optimization terminated successfully.
         Current function value: 0.954904
         Iterations 6


### IGT

In [158]:
coef = model.params[0]['leg_length_z_score']
leg_length_IGT_Model2_effect = np.exp(coef*(-std_dev))

ci_df = model.conf_int().reset_index()

leg_length_IGT_Model2_upper = np.exp(ci_df.loc[(ci_df['GT_Category_mapped'] == '2') & (ci_df['level_1'] == 'leg_length_z_score'), 'upper'].values[0] * (-std_dev))
leg_length_IGT_Model2_lower = np.exp(ci_df.loc[(ci_df['GT_Category_mapped'] == '2') & (ci_df['level_1'] == 'leg_length_z_score'), 'lower'].values[0] * (-std_dev))

leg_length_IGT_model2 = f"{abs(leg_length_IGT_Model2_effect):.2f} ({leg_length_IGT_Model2_upper:.2f}-{leg_length_IGT_Model2_lower:.2f})"
print(leg_length_IGT_model2)

1.08 (1.01-1.15)


### Diabetes

In [159]:
coef = model.params[1]['leg_length_z_score']
leg_length_Diabetes_Model2_effect = np.exp(coef*(-std_dev))

ci_df = model.conf_int().reset_index()

leg_length_Diabetes_Model2_upper = np.exp(ci_df.loc[(ci_df['GT_Category_mapped'] == '3') & (ci_df['level_1'] == 'leg_length_z_score'), 'upper'].values[0] * (-std_dev))
leg_length_Diabetes_Model2_lower = np.exp(ci_df.loc[(ci_df['GT_Category_mapped'] == '3') & (ci_df['level_1'] == 'leg_length_z_score'), 'lower'].values[0] * (-std_dev))

leg_length_Diabetes_Model2 = f"{abs(leg_length_Diabetes_Model1_effect):.2f} ({leg_length_Diabetes_Model2_upper:.2f}-{leg_length_Diabetes_Model2_lower:.2f})"
print(leg_length_Diabetes_Model2)

1.17 (1.05-1.21)


## Model 3

In [160]:
model = mnlogit('GT_Category_mapped ~ leg_length_z_score+HSAGEIR+parental_diabetes+physical_activity+education_12_years+income_20000_or_more+smoking_hist+MAPF2+PBF+PBF*Sex',data=ogtt_df).fit()

Optimization terminated successfully.
         Current function value: 0.946548
         Iterations 6


### IGT

In [161]:
coef = model.params[0]['leg_length_z_score']
leg_length_IGT_Model3_effect = np.exp(coef*(-std_dev))

ci_df = model.conf_int().reset_index()

leg_length_IGT_Model3_upper = np.exp(ci_df.loc[(ci_df['GT_Category_mapped'] == '2') & (ci_df['level_1'] == 'leg_length_z_score'), 'upper'].values[0] * (-std_dev))
leg_length_IGT_Model3_lower = np.exp(ci_df.loc[(ci_df['GT_Category_mapped'] == '2') & (ci_df['level_1'] == 'leg_length_z_score'), 'lower'].values[0] * (-std_dev))

leg_length_IGT_model3 = f"{abs(leg_length_IGT_Model3_effect):.2f} ({leg_length_IGT_Model3_upper:.2f}-{leg_length_IGT_Model3_lower:.2f})"
print(leg_length_IGT_model3)

1.07 (1.00-1.13)


### Diabetes

In [162]:
coef = model.params[1]['leg_length_z_score']
leg_length_Diabetes_Model3_effect = np.exp(coef*(-std_dev))

ci_df = model.conf_int().reset_index()

leg_length_Diabetes_Model3_upper = np.exp(ci_df.loc[(ci_df['GT_Category_mapped'] == '3') & (ci_df['level_1'] == 'leg_length_z_score'), 'upper'].values[0] * (-std_dev))
leg_length_Diabetes_Model3_lower = np.exp(ci_df.loc[(ci_df['GT_Category_mapped'] == '3') & (ci_df['level_1'] == 'leg_length_z_score'), 'lower'].values[0] * (-std_dev))

leg_length_Diabetes_Model3 = f"{abs(leg_length_Diabetes_Model3_effect):.2f} ({leg_length_Diabetes_Model3_upper:.2f}-{leg_length_Diabetes_Model3_lower:.2f})"
print(leg_length_Diabetes_Model3)

1.11 (1.03-1.20)


## Leg Length to height ratio

In [163]:
std_dev = std_devs['leg_length_to_height_ratio_z_score']
anthropometric_measures = [['leg_length_to_height_ratio_z_score',0],['leg_length_to_height_ratio_z_score',1]]

## Model 1

In [164]:
model = mnlogit('GT_Category_mapped ~ leg_length_to_height_ratio_z_score+HSAGEIR',data=ogtt_df).fit()

Optimization terminated successfully.
         Current function value: 0.987131
         Iterations 5


### IGT

In [165]:
coef = model.params[0]['leg_length_to_height_ratio_z_score']
ratio_IGT_Model1_effect = np.exp(coef*(-std_dev))

ci_df = model.conf_int().reset_index()

ratio_IGT_Model1_upper = np.exp(ci_df.loc[(ci_df['GT_Category_mapped'] == '2') & (ci_df['level_1'] == 'leg_length_to_height_ratio_z_score'), 'upper'].values[0] * (-std_dev))
ratio_IGT_Model1_lower = np.exp(ci_df.loc[(ci_df['GT_Category_mapped'] == '2') & (ci_df['level_1'] == 'leg_length_to_height_ratio_z_score'), 'lower'].values[0] * (-std_dev))

ratio_IGT_model1 = f"{abs(ratio_IGT_Model1_effect):.2f} ({ratio_IGT_Model1_upper:.2f}-{ratio_IGT_Model1_lower:.2f})"
print(ratio_IGT_model1)

1.07 (1.00-1.13)


### Diabetes

In [166]:
coef = model.params[1]['leg_length_to_height_ratio_z_score']
ratio_Diabetes_Model1_effect = np.exp(coef*(-std_dev))

ci_df = model.conf_int().reset_index()

ratio_Diabetes_Model1_upper = np.exp(ci_df.loc[(ci_df['GT_Category_mapped'] == '3') & (ci_df['level_1'] == 'leg_length_to_height_ratio_z_score'), 'upper'].values[0] * (-std_dev))
ratio_Diabetes_Model1_lower = np.exp(ci_df.loc[(ci_df['GT_Category_mapped'] == '3') & (ci_df['level_1'] == 'leg_length_to_height_ratio_z_score'), 'lower'].values[0] * (-std_dev))

ratio_Diabetes_Model1 = f"{abs(ratio_Diabetes_Model1_effect):.2f} ({ratio_Diabetes_Model1_upper:.2f}-{ratio_Diabetes_Model1_lower:.2f})"
print(ratio_Diabetes_Model1)

1.18 (1.10-1.27)


## Model 2

In [167]:
model = mnlogit('GT_Category_mapped ~ leg_length_to_height_ratio_z_score+HSAGEIR+parental_diabetes+physical_activity+education_12_years+income_20000_or_more+smoking_hist+MAPF2',data=ogtt_df).fit()

Optimization terminated successfully.
         Current function value: 0.954454
         Iterations 6


### IGT

In [168]:
coef = model.params[0]['leg_length_to_height_ratio_z_score']
ratio_IGT_Model2_effect = np.exp(coef*(-std_dev))

ci_df = model.conf_int().reset_index()

ratio_IGT_Model2_upper = np.exp(ci_df.loc[(ci_df['GT_Category_mapped'] == '2') & (ci_df['level_1'] == 'leg_length_to_height_ratio_z_score'), 'upper'].values[0] * (-std_dev))
ratio_IGT_Model2_lower = np.exp(ci_df.loc[(ci_df['GT_Category_mapped'] == '2') & (ci_df['level_1'] == 'leg_length_to_height_ratio_z_score'), 'lower'].values[0] * (-std_dev))

ratio_IGT_model2 = f"{abs(ratio_IGT_Model2_effect):.2f} ({ratio_IGT_Model2_upper:.2f}-{ratio_IGT_Model2_lower:.2f})"
print(ratio_IGT_model2)

1.06 (0.99-1.12)


### Diabetes

In [169]:
coef = model.params[1]['leg_length_to_height_ratio_z_score']
ratio_Diabetes_Model2_effect = np.exp(coef*(-std_dev))

ci_df = model.conf_int().reset_index()

ratio_Diabetes_Model2_upper = np.exp(ci_df.loc[(ci_df['GT_Category_mapped'] == '3') & (ci_df['level_1'] == 'leg_length_to_height_ratio_z_score'), 'upper'].values[0] * (-std_dev))
ratio_Diabetes_Model2_lower = np.exp(ci_df.loc[(ci_df['GT_Category_mapped'] == '3') & (ci_df['level_1'] == 'leg_length_to_height_ratio_z_score'), 'lower'].values[0] * (-std_dev))

ratio_Diabetes_Model2 = f"{abs(ratio_Diabetes_Model2_effect):.2f} ({ratio_Diabetes_Model2_upper:.2f}-{ratio_Diabetes_Model2_lower:.2f})"
print(ratio_Diabetes_Model2)

1.17 (1.09-1.26)


## Model 3

In [170]:
model = mnlogit('GT_Category_mapped ~ leg_length_to_height_ratio_z_score+HSAGEIR+parental_diabetes+physical_activity+education_12_years+income_20000_or_more+smoking_hist+MAPF2+PBF+PBF*Sex',data=ogtt_df).fit()

Optimization terminated successfully.
         Current function value: 0.946122
         Iterations 6


### IGT

In [171]:
coef = model.params[0]['leg_length_to_height_ratio_z_score']
ratio_IGT_Model3_effect = np.exp(coef*(-std_dev))

ci_df = model.conf_int().reset_index()

ratio_IGT_Model3_upper = np.exp(ci_df.loc[(ci_df['GT_Category_mapped'] == '2') & (ci_df['level_1'] == 'leg_length_to_height_ratio_z_score'), 'upper'].values[0] * (-std_dev))
ratio_IGT_Model3_lower = np.exp(ci_df.loc[(ci_df['GT_Category_mapped'] == '2') & (ci_df['level_1'] == 'leg_length_to_height_ratio_z_score'), 'lower'].values[0] * (-std_dev))

ratio_IGT_model3 = f"{abs(ratio_IGT_Model3_effect):.2f} ({ratio_IGT_Model3_upper:.2f}-{ratio_IGT_Model3_lower:.2f})"
print(ratio_IGT_model3)

1.05 (0.98-1.11)


### Diabetes

In [172]:
coef = model.params[1]['leg_length_to_height_ratio_z_score']
ratio_Diabetes_Model3_effect = np.exp(coef*(-std_dev))

ci_df = model.conf_int().reset_index()

ratio_Diabetes_Model3_upper = np.exp(ci_df.loc[(ci_df['GT_Category_mapped'] == '3') & (ci_df['level_1'] == 'leg_length_to_height_ratio_z_score'), 'upper'].values[0] * (-std_dev))
ratio_Diabetes_Model3_lower = np.exp(ci_df.loc[(ci_df['GT_Category_mapped'] == '3') & (ci_df['level_1'] == 'leg_length_to_height_ratio_z_score'), 'lower'].values[0] * (-std_dev))

ratio_Diabetes_Model3 = f"{abs(ratio_Diabetes_Model3_effect):.2f} ({ratio_Diabetes_Model3_upper:.2f}-{ratio_Diabetes_Model3_lower:.2f})"
print(ratio_Diabetes_Model3)

1.15 (1.07-1.24)


# Final Table

In [173]:
data = {
    'Male': [height_male_model1, height_male_model2, '-', leg_length_male_model1, leg_length_male_model2, '-', ratio_male_model1, ratio_male_model2, '-'],
    'Female': [height_female_model1, height_female_model2, '-', leg_length_female_model1, leg_length_female_model2, '-', ratio_female_model1, ratio_female_model2, '-'],
    'HOMA_IR': [height_HOMA_model1, height_HOMA_model2, height_HOMA_model3, leg_length_HOMA_model1, leg_length_HOMA_model2, leg_length_HOMA_model3, ratio_HOMA_model1, ratio_HOMA_model2, ratio_HOMA_model3],
    'IGT': [height_IGT_model1, height_IGT_model2, height_IGT_model3, leg_length_IGT_model1, leg_length_IGT_model2, leg_length_IGT_model3, ratio_IGT_model1, ratio_IGT_model2, ratio_IGT_model3],
    'Diabetes': [height_diabetes_model1, height_diabetes_model2, height_diabetes_model3, leg_length_diabetes_model1, leg_length_Diabetes_Model2, leg_length_Diabetes_Model3, ratio_Diabetes_Model1, ratio_Diabetes_Model2, ratio_Diabetes_Model3]
}

In [174]:
df = pd.DataFrame(data)
df

,Male,Female,HOMA_IR,IGT,Diabetes
0,+0.18 (-0.02-0.38),+0.80 (0.42-1.17),1.00 (0.98-1.01),1.12 (1.05-1.19),1.10 (1.03-1.18)
1,+0.15 (-0.05-0.35),+0.60 (0.23-0.97),0.99 (0.98-1.01),1.10 (1.03-1.17),1.06 (0.98-1.14)
2,-,-,0.98 (0.97-0.99),1.09 (1.02-1.16),1.05 (0.97-1.13)
3,+0.17 (-0.03-0.37),+1.08 (0.71-1.45),1.02 (1.01-1.04),1.10 (1.03-1.17),1.17 (1.09-1.25)
4,+0.13 (-0.07-0.33),+0.86 (0.49-1.23),1.02 (1.00-1.03),1.08 (1.01-1.15),1.17 (1.05-1.21)
5,-,-,1.00 (0.99-1.01),1.07 (1.00-1.13),1.11 (1.03-1.20)
6,+0.24 (0.04-0.44),+1.11 (0.77-1.45),1.06 (1.04-1.07),1.07 (1.00-1.13),1.18 (1.10-1.27)
7,+0.20 (0.00-0.40),+0.98 (0.64-1.32),1.05 (1.04-1.07),1.06 (0.99-1.12),1.17 (1.09-1.26)
8,-,-,1.04 (1.02-1.05),1.05 (0.98-1.11),1.15 (1.07-1.24)


In [175]:
df.to_csv('FINAL_TABLE.csv')